In [1]:
import tensorflow
import numpy as np
import keras
from keras.datasets import mnist
import matplotlib.pyplot as plt
from keras.models import Sequential

In [2]:
def define_discriminator(in_shape = (28, 28, 1)):
    model = Sequential()
    model.add(keras.layers.Conv2D(40, (5, 5), strides = (1, 1), padding = 'valid'))
    model.add(keras.layers.LeakyReLU(alpha = 0.2))
    model.add(keras.layers.Dropout(0.4))
    model.add(keras.layers.Conv2D(60, (3, 3), strides = (1, 1), padding = 'valid'))
    model.add(keras.layers.LeakyReLU(alpha = 0.2))
    model.add(keras.layers.Dropout(0.4))
    model.add(keras.layers.Conv2D(80, (3, 3), strides = (1, 1), padding = 'valid'))
    model.add(keras.layers.LeakyReLU(alpha = 0.2))
    model.add(keras.layers.Dropout(0.4))
    model.add(keras.layers.Conv2D(100, (3, 3), strides = (1, 1), padding = 'valid'))
    model.add(keras.layers.LeakyReLU(alpha = 0.2))
    model.add(keras.layers.Dropout(0.4))
    model.add(keras.layers.Conv2D(120, (3, 3), strides = (1, 1), padding = 'valid'))
    model.add(keras.layers.LeakyReLU(alpha = 0.2))
    model.add(keras.layers.Dropout(0.4))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(1, activation = 'sigmoid'))
    
    model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return model

In [3]:
def define_generator(latent_dim):
    gen_model = Sequential()
    nodes = 128 * 7 * 7
    gen_model.add(keras.layers.Dense(nodes, input_dim = latent_dim))
    gen_model.add(keras.layers.LeakyReLU(alpha = 0.2))
    gen_model.add(keras.layers.Reshape((7, 7, 128)))
    gen_model.add(keras.layers.Conv2DTranspose(128, (4, 4), strides = (2, 2), padding = 'valid'))
    gen_model.add(keras.layers.LeakyReLU(alpha = 0.2))
    
    gen_model.add(keras.layers.Conv2DTranspose(128, (4, 4), strides = (2, 2), padding = 'valid'))
    gen_model.add(keras.layers.LeakyReLU(alpha = 0.2))
    
    gen_model.add(keras.layers.Conv2D(1, (7, 7), padding = 'valid', activation = 'sigmoid'))
    
    return gen_model

In [4]:
def define_gans(g_model, d_model):
    d_model.trainable = False
    model = Sequential()
    model.add(g_model)
    model.add(d_model)
    
    model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model

In [5]:
def load_real_samples():
    (trainX, _), (_, _) = mnist.load_data()
    X = np.expand_dims(trainX, axis = -1)
    X = X/255
    return X

In [6]:
def generate_real_samples(dataset, n_samples):
    idx = np.random.randint(0, dataset.shape[0], n_samples)
    X = dataset[idx]
    Y = np.ones((n_samples, 1))
    return X, Y

In [7]:
def generate_latent_points(latent_dim, n_samples):
    X = np.random.randn(latent_dim * n_samples)
    X = X.reshape(n_samples, latent_dim)
    return X

In [8]:
def generate_fake_samples(gen_model, latent_dim, n_samples):
    X_input = generate_latent_points(latent_dim, n_samples)
    p_g = gen_model.predict(X_input)
    Y = np.zeros((n_samples, 1))
    return p_g, Y

In [9]:
def viz_plot(example, epoch, n = 10):
    for i in range(n * n):
        plt.subplot(n, n , i + 1)
        plt.imshow(example[i, :, :, :], cmap = 'gray')
    file_name = f'C:/Users/bhatt/OneDrive/Desktop/whisper-main/DS_PP_Aug_23/Machine Learning/{epoch}_epoch.jpg'
    plt.savefig(file_name)
    
    return plt

In [10]:
def architecture(g_model, d_model, dataset, latent_dim, epoch, batch = 100):
    X_real, Y_real = generate_real_samples(dataset, batch)
    
    _, acc_real = d_model.evaluate(X_real, Y_real)
    
    X_fake, Y_fake = generate_fake_samples(g_model, latent_dim, batch)
    
    _, acc_fake = d_model.evaluate(X_fake, Y_fake)
    
    print(f'Accuracy real: {acc_real}, Accuracy fake: {acc_fake}')
    
    plot = viz_plot(X_fake, epoch)

In [11]:
def train(g_model, d_model, gan_model, dataset, latent_dim, epoch, batch_size = 100):
    batch_per_epoch = int(dataset.shape[0] / batch_size)
    half_batch = int(batch_size / 2)
    
    for i in range(epoch):
        for j in range(batch_per_epoch):
            X_real, Y_real = generate_real_samples(dataset, half_batch)
            
            X_fake, Y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            
            d_loss_real, _ = d_model.train_on_batch(X_real, Y_real)
            d_loss_fake, _ = d_model.train_on_batch(X_fake, Y_fake)
            
            d_loss = d_loss_fake + d_loss_real
            
            X_noise = generate_latent_points(latent_dim, batch_size)
            
            Y_noise = np.ones((batch_size, 1))
            
            g_loss, _ = gan_model.train_on_batch(X_noise, Y_noise)
            
            print(f'{i}, Dloss: {d_loss}, Gloss: {g_loss}')
            
        if i % 5 == 0:
            architecture(g_model, d_model, dataset, latent_dim, i)

In [12]:
latent_dim = 100

d_model = define_discriminator()

g_model = define_generator(latent_dim)

gan_model = define_gans(g_model, d_model)

dataset = load_real_samples()

train(g_model, d_model, gan_model, dataset, latent_dim, 100)

2/2 [==============================] - 0s 36ms/step
0, Dloss: 1.8726101517677307, Gloss: 0.6256900429725647
2/2 [==============================] - 0s 36ms/step
0, Dloss: 1.3198096752166748, Gloss: 0.701171875
2/2 [==============================] - 0s 37ms/step
0, Dloss: 1.3511700630187988, Gloss: 0.7246828675270081
2/2 [==============================] - 0s 29ms/step
0, Dloss: 1.3271278738975525, Gloss: 0.7525061964988708
2/2 [==============================] - 0s 29ms/step
0, Dloss: 1.2442890405654907, Gloss: 0.8008676171302795
2/2 [==============================] - 0s 32ms/step
0, Dloss: 0.9918825626373291, Gloss: 0.922301173210144
2/2 [==============================] - 0s 35ms/step
0, Dloss: 0.5073358379304409, Gloss: 1.493152141571045
2/2 [==============================] - 0s 36ms/step
0, Dloss: 0.13061576418112963, Gloss: 3.6545238494873047
2/2 [==============================] - 0s 30ms/step
0, Dloss: 0.004667037515901029, Gloss: 8.216655731201172
2/2 [==============================

2/2 [==============================] - 0s 26ms/step
0, Dloss: 6.1869907770754386e-06, Gloss: 11.996163368225098
2/2 [==============================] - 0s 26ms/step
0, Dloss: 6.809365512339127e-06, Gloss: 12.03816032409668
2/2 [==============================] - 0s 32ms/step
0, Dloss: 6.311498180755493e-06, Gloss: 11.983699798583984
2/2 [==============================] - 0s 27ms/step
0, Dloss: 6.234757165657356e-06, Gloss: 12.03735637664795
2/2 [==============================] - 0s 38ms/step
0, Dloss: 6.097473398910397e-06, Gloss: 12.038063049316406
2/2 [==============================] - 0s 28ms/step
0, Dloss: 6.703736167096297e-06, Gloss: 11.990987777709961
2/2 [==============================] - 0s 29ms/step
0, Dloss: 6.173900021871209e-06, Gloss: 11.999885559082031
2/2 [==============================] - 0s 27ms/step
0, Dloss: 9.987355952034704e-06, Gloss: 12.017155647277832
2/2 [==============================] - 0s 32ms/step
0, Dloss: 5.963123885344146e-06, Gloss: 11.962535858154297
2/

2/2 [==============================] - 0s 29ms/step
0, Dloss: 8.115737374669516e-06, Gloss: 11.779674530029297
2/2 [==============================] - 0s 28ms/step
0, Dloss: 8.653344307940625e-06, Gloss: 11.793266296386719
2/2 [==============================] - 0s 28ms/step
0, Dloss: 8.268088974666687e-06, Gloss: 11.811687469482422
2/2 [==============================] - 0s 33ms/step
0, Dloss: 7.442616289425621e-06, Gloss: 11.732440948486328
2/2 [==============================] - 0s 28ms/step
0, Dloss: 8.374429188329202e-06, Gloss: 11.777731895446777
2/2 [==============================] - 0s 28ms/step
0, Dloss: 8.059925421743941e-06, Gloss: 11.803579330444336
2/2 [==============================] - 0s 29ms/step
0, Dloss: 8.23147886808151e-06, Gloss: 11.7744722366333
2/2 [==============================] - 0s 28ms/step
0, Dloss: 8.578212023554067e-06, Gloss: 11.823037147521973
2/2 [==============================] - 0s 26ms/step
0, Dloss: 8.690356587237602e-06, Gloss: 11.815895080566406
2/2 

2/2 [==============================] - 0s 29ms/step
0, Dloss: 6.565561033511358e-06, Gloss: 11.941637992858887
2/2 [==============================] - 0s 33ms/step
0, Dloss: 6.541765337431116e-06, Gloss: 11.9776611328125
2/2 [==============================] - 0s 27ms/step
0, Dloss: 6.8887448144124355e-06, Gloss: 11.974225997924805
2/2 [==============================] - 0s 33ms/step
0, Dloss: 6.712139343697035e-06, Gloss: 11.984513282775879
2/2 [==============================] - 0s 31ms/step
0, Dloss: 6.76361779627932e-06, Gloss: 11.961298942565918
2/2 [==============================] - 0s 30ms/step
0, Dloss: 6.666142354128114e-06, Gloss: 11.989724159240723
2/2 [==============================] - 0s 27ms/step
0, Dloss: 6.929090085806139e-06, Gloss: 11.970017433166504
2/2 [==============================] - 0s 29ms/step
0, Dloss: 6.6869124921709044e-06, Gloss: 11.966793060302734
2/2 [==============================] - 0s 28ms/step
0, Dloss: 6.94452755567454e-06, Gloss: 11.959912300109863
2/2

2/2 [==============================] - 0s 35ms/step
0, Dloss: 5.3080172417868665e-06, Gloss: 12.221757888793945
2/2 [==============================] - 0s 34ms/step
0, Dloss: 7.069362709444249e-06, Gloss: 12.218591690063477
2/2 [==============================] - 0s 36ms/step
0, Dloss: 5.6447540970339105e-06, Gloss: 12.19186019897461
2/2 [==============================] - 0s 27ms/step
0, Dloss: 5.483278407685931e-06, Gloss: 12.191143035888672
2/2 [==============================] - 0s 31ms/step
0, Dloss: 5.065507507383618e-06, Gloss: 12.281048774719238
2/2 [==============================] - 0s 28ms/step
0, Dloss: 5.458195371454977e-06, Gloss: 12.216861724853516
2/2 [==============================] - 0s 33ms/step
0, Dloss: 5.338014138045466e-06, Gloss: 12.227485656738281
2/2 [==============================] - 0s 32ms/step
0, Dloss: 5.275194368159486e-06, Gloss: 12.225083351135254
2/2 [==============================] - 0s 28ms/step
0, Dloss: 5.000519416153981e-06, Gloss: 12.20962905883789
2

...

2/2 [==============================] - 0s 57ms/step
3, Dloss: 3.0786850901255436e-07, Gloss: 14.970807075500488
2/2 [==============================] - 0s 53ms/step
3, Dloss: 3.2142762072281814e-07, Gloss: 15.02168083190918
2/2 [==============================] - 0s 62ms/step
3, Dloss: 3.061489206885454e-07, Gloss: 15.044757843017578
2/2 [==============================] - 0s 49ms/step
3, Dloss: 3.384256219996201e-07, Gloss: 15.004591941833496
2/2 [==============================] - 0s 45ms/step
3, Dloss: 3.082277544830903e-07, Gloss: 15.060325622558594
2/2 [==============================] - 0s 42ms/step
3, Dloss: 3.2206460751703727e-07, Gloss: 15.062225341796875
2/2 [==============================] - 0s 46ms/step
3, Dloss: 3.0283502688854824e-07, Gloss: 15.023638725280762
2/2 [==============================] - 0s 44ms/step
3, Dloss: 3.4727091934210063e-07, Gloss: 15.056836128234863
2/2 [==============================] - 0s 44ms/step
3, Dloss: 3.0950863633139036e-07, Gloss: 15.015261650085

2/2 [==============================] - 0s 26ms/step
3, Dloss: 2.679757359381051e-07, Gloss: 15.197663307189941
2/2 [==============================] - 0s 26ms/step
3, Dloss: 2.5580553202454427e-07, Gloss: 15.115043640136719
2/2 [==============================] - 0s 31ms/step
3, Dloss: 3.0499933239896623e-07, Gloss: 15.201986312866211
2/2 [==============================] - 0s 28ms/step
3, Dloss: 2.9119476252503773e-07, Gloss: 15.225471496582031
2/2 [==============================] - 0s 51ms/step
3, Dloss: 2.734367412668863e-07, Gloss: 15.26645278930664
2/2 [==============================] - 0s 74ms/step
3, Dloss: 2.8033363557361307e-07, Gloss: 15.19324016571045
2/2 [==============================] - 0s 45ms/step
3, Dloss: 2.8268628425684884e-07, Gloss: 15.166600227355957
2/2 [==============================] - 0s 44ms/step
3, Dloss: 2.8481678100427413e-07, Gloss: 15.228157997131348
2/2 [==============================] - 0s 52ms/step
3, Dloss: 2.881010604319223e-07, Gloss: 15.1821165084838


KeyboardInterrupt



Error in callback <function flush_figures at 0x0000011484511670> (for post_execute):


KeyboardInterrupt: 